In [8]:
using DataFrames
using Plots

df = readtable("train.csv");

# Variablen-Typen
Für statistische Variablen lassen sich gewisse Kenngrössen bestimmen welche die 
**allgemeine/durchschnittliche Tendenz**, die **Streuung** oder die **Form** der zugrundeliegenden
Verteilung beschreiben. Je nach dem welche von diesen Grössen für den konkreten Fall (=Variable)
wohldefiniert sind, kann man zwischen 3 Haupttypen unterscheiden.

### Allgemeine Tendenz
- Arithmetisches Mittel: $\bar{x} = \frac{1}{n}\sum_{i=1}^n x_i$
- Median: Es gibt $\frac{n}{2}$ Werte die Grösser sind als der Median und $\frac{n}{2}$ die kleiner sind.
- Mode: Der am häufigsten vorkommende Wert.

### Streuung
- Gesamter Wertebereich (min bis max)
- Quantile (die ersten, zweiten, letzten.. x-ten Prozent)
- Varianz: $\sigma^2 = \frac{1}{n}\sum_{i=1}^n (x_i - \bar{x})^2$
- Standardabweichung: $\sigma = \sqrt{\sigma^2}$

### Form
- Skewness (Schiefe): Wie symmetrisch ist die Verteilung? $\frac{1}{n}\sum_{i=1}^n (x_i - \bar{x})^3$
- Kurtosis (Wölbung): Wie spitzig ist die Verteilung $\frac{1}{n}\sum_{i=1}^n (x_i - \bar{x})^4$

# Quantitative Variablen
Lassen Berechnungen für alle oben genannten Kenngrössen zu und
werden in der Regel durch Reale Zahlen dargestellt, wie z.B:
- Körpergewicht in kg
- Temperatur in Grad Kelvin
- Kokainverbrauch pro Person pro Jahr
- Anzahl Einwohner

Weiter liegt Quantitativen Variablen eine sinnvolle numerische Skala zugrunde auf der sich eine
"Distanz" berechnen lässt. Als Beispiel ist es sinnvoll zu sagen 2000 kg sind 4 mal so viel wie 500 kg.

**Alle oben genannten Kenngrössen sind wohldefiniert.**

# Ordinale Variablen
Haben keine sinnvolle nuerische Skala als Grundlage, es gibt aber eine eindeutig definierte Reihenfolge der Elemente. Man kann also Elemente sortieren, aber keine sinnvolle euklidische Distanz zwischen ihen Bestimmen.
- Hausnummern
- T-shirt Grössen
- Akunto Abo-Preise

**Definierte Kenngrössen: Reihenfolge, Median, Mode, Quantile**

# Kategorische Variablen
Es lassen sich weder eine Reihenfolge, noch eine Distanz zwischen Elementen sinnvoll definieren.

- Farben
- Vornamen
- Nationalität

**Definierte Kenngrössen: Mode**






In [9]:
# Untersuche nochmals die Daten und Bestimme den Typ für jede Variable:
names(df)

12-element Array{Symbol,1}:
 :PassengerId
 :Survived   
 :Pclass     
 :Name       
 :Sex        
 :Age        
 :SibSp      
 :Parch      
 :Ticket     
 :Fare       
 :Cabin      
 :Embarked   

In [10]:
function onehot!(df, varname)
    for keyword in unique(df[:, varname])
        sym_keyword = Symbol(keyword)
        df[sym_keyword] = zeros(Int, size(df, 1))
        for i in 1:size(df, 1)
            if df[i, varname] == keyword
                df[i, sym_keyword] = 1
            end
        end
    end
end

function binarize!(df, varname)
    tmp = zeros(Int, size(df, 1))
    for (i, keyword) in enumerate(unique(df[varname]))
        flt = df[varname] .== keyword
        tmp[flt] = i-1
    end
    df[varname] = tmp
end

binarize! (generic function with 1 method)

In [11]:
df[isna(df[:Embarked]), :Embarked] = "S"
onehot!(df, :Embarked)
binarize!(df, :Sex);

In [12]:
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,S,C,Q
1,1,0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.25,NA,S,1,0,0
2,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",1,38.0,1,0,PC 17599,71.2833,C85,C,0,1,0
3,3,1,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.925,NA,S,1,0,0
4,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1,C123,S,1,0,0
5,5,0,3,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.05,NA,S,1,0,0
6,6,0,3,"Moran, Mr. James",0,NA,0,0,330877,8.4583,NA,Q,0,0,1
7,7,0,1,"McCarthy, Mr. Timothy J",0,54.0,0,0,17463,51.8625,E46,S,1,0,0
8,8,0,3,"Palsson, Master. Gosta Leonard",0,2.0,3,1,349909,21.075,NA,S,1,0,0
9,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",1,27.0,0,2,347742,11.1333,NA,S,1,0,0
10,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",1,14.0,1,0,237736,30.0708,NA,C,0,1,0


In [26]:
@show(mean(dropna(df[:Age])))
@show(median(dropna(df[:Age])))
@show(mode(dropna(df[:Age])))
histogram(dropna(df[:Age]), bins=0:4:80, show=true)


mean(dropna(df[:Age])) = 29.69911764705882
median(dropna(df[:Age])) = 28.0
mode(dropna(df[:Age])) = 24.0


In [28]:
agenew = copy(df[:Age]);
flt = isna(agenew)
agenew[flt] = 30

30

In [36]:
df[flt,:]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,S,C,Q
1,6,0,3,"Moran, Mr. James",0,NA,0,0,330877,8.4583,NA,Q,0,0,1
2,18,1,2,"Williams, Mr. Charles Eugene",0,NA,0,0,244373,13.0,NA,S,1,0,0
3,20,1,3,"Masselmani, Mrs. Fatima",1,NA,0,0,2649,7.225,NA,C,0,1,0
4,27,0,3,"Emir, Mr. Farred Chehab",0,NA,0,0,2631,7.225,NA,C,0,1,0
5,29,1,3,"O'Dwyer, Miss. Ellen ""Nellie""",1,NA,0,0,330959,7.8792,NA,Q,0,0,1
6,30,0,3,"Todoroff, Mr. Lalio",0,NA,0,0,349216,7.8958,NA,S,1,0,0
7,32,1,1,"Spencer, Mrs. William Augustus (Marie Eugenie)",1,NA,1,0,PC 17569,146.5208,B78,C,0,1,0
8,33,1,3,"Glynn, Miss. Mary Agatha",1,NA,0,0,335677,7.75,NA,Q,0,0,1
9,37,1,3,"Mamee, Mr. Hanna",0,NA,0,0,2677,7.2292,NA,C,0,1,0
10,43,0,3,"Kraeff, Mr. Theodor",0,NA,0,0,349253,7.8958,NA,C,0,1,0


In [33]:
sum(flt)

177

In [34]:
length(flt)

891